In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.plotly as py
import seaborn as sns

In [2]:
#def load_data():
global smart
global gas
global weather
df =pd.read_csv("/datc/opschaler/smartmeter_data/P01S01W0373.csv",header=0,delimiter=";",parse_dates=['Timestamp', 'gasTimestamp'])

# Split the smartmeter and gasmeter data
smart = df.iloc[:,:7]
gas = df.iloc[:, 7:]

In [3]:
# Change all datetime columns to the same column name
smart = smart.rename(index=str, columns={"Timestamp":"datetime"})
gas = gas.rename(index=str, columns={"gasTimestamp":"datetime"})

#df = df[['datetime', 'eMeter', 'eMeterLow', 'ePower', 'gasMeter', 'T', 'Q']]
#df = df[['datetime', 'eMeter', 'eMeterLow', 'ePower', 'gasMeter']]
smart.head()

# Set datetime as index
smart = smart.set_index(['datetime'])
gas = gas.set_index(['datetime'])

In [4]:
"""
Resample the dataframes to 10s intervals.
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.interpolate.html#pandas.DataFrame.interpolate
"""
smart = smart.resample('10s').mean()
gas = gas.resample('H').mean() # Does not look nice, but it works

In [ ]:
#These columns should be created in the end: column_names = ['House_ID','Amount_of_NaNs','Max_consecutive_NaNs', 'Interpolated','Days','File_size','Usable']
#Create a df to store max consecutive NaNs
info_data = pd.DataFrame(columns={'House_ID','File_size','Initial_Date','Final_Date','Days','Amount_of_NaNs','Max_Consecutive_NaNs'})
tempstring = 'P0123423.csv'

def extract_info(df,dwelling_id='test.csv'):

    
    #Process info to get max gap size
    df_nan=df.isnull().any(axis=1)*1
    temp = pd.DataFrame()
    temp['temp']=df_nan.groupby((df_nan != df_nan.shift()).cumsum()).transform('size') * df_nan
    max_gap=temp['temp'].max()
    
    
    
    #Extract information about dates and number of NaNs
    initial_date_string= df.index[0].strftime('%Y-%m-%d %H:%M:%S')
    final_date_string= df.index[-1].strftime('%Y-%m-%d %H:%M:%S')
    days=df.index[-1]-df.index[0]
    NaN_count=df.isnull().sum()[0]
    
    #Append all information to the global info_data dataframe
    info_data.loc[len(info_data.index)] = [dwelling_id, df.memory_usage(index=True).sum()/1000000, initial_date_string, final_date_string, days, NaN_count, max_gap]

Empty DataFrame
Columns: [Days, House_ID, Final_Date, File_size, Initial_Date, Max_Consecutive_NaNs, Amount_of_NaNs]
Index: []


In [ ]:
def  df_NaN_checker(df, threshold_percentage):
    """
    Input:
    df = Pandas dataframe
    threshold_percentage = The minimun percentage a NaN streak should have to be shown in the output df of this function.
    
    This function has a Pandas dataframe as input. 
    It checks per column the amount of NaNs which are behind each other and outputs the relevant information into a Pandas dataframe.
    The output df is in the format:
    Column name, Start index, Stop index, Amount of NaNs
    
    Where'
    'Column name' is the column name of the original df, where the NaN streak is in.
    'Start index' is the start index value of that NaN streak.
    'Stop index' is the stop index value of that NaN streak
    'Amount of NaNs' is the amount of NaNs behind each other, in other word the length of the NaN streak.
    """
    columns = df.columns
    output = []
    df_temp 

    for i in range(len(columns)):
        column_name = columns[i]
    
        column_info_final = []
        column_info = []
        temp = []
        x = False
    
        for j,value in enumerate(df[column_name]):
            if x == False and value == True:
                temp.append(j)
                x = True
            elif x == True and value == True:
                temp.append(j)
            elif x == True and value == False:
                column_info.append(temp)
                temp = []
                x = False
        
        lengths = []

        for array in column_info:
            lengths.append([array[0], array[-1], len(array)])
    
        output.append(lengths)
    

    # Convert df_info to a readable dataframe instead of list

    """
    Row per column from the 'output' list
    Columns: start-index, stop-index, NaN streak
    """
    columns = df.columns

    df_info = pd.DataFrame(columns=['Column name','Start index', 'Stop index', 'Amount of NaNs'])

    for column in range(len(output)):
        for i in range(len(output[column])):
            column_name = df.columns[column]
            start = output[column][i][0]
            stop = output[column][i][1]
            amount = output[column][i][2]
            df_info = df_info.append({'Column name':column_name,'Start index':start, 'Stop index': stop, 'Amount of NaNs':amount}, ignore_index=True)
        pass

    percentage = (df_info['Amount of NaNs']/len(df))*100

    df_info.drop(df_info[percentage < threshold_percentage].index, inplace=True)

    return df_info

In [ ]:
df_NaN_checker(smart,50)